## Recommendation systems

Built a recommendation system from scratch using collaborative filtering with matrix factorization and predict the movie ratings that MovieLens users give to movies.

This dataset contains 100836 ratings, 610 users and 9724 movies.

### Cold Start Problem

The colaborative filtering method discussed here does address the problem of new user or new movies. What prediction would I use in these cases:

- A new user but a known movie
- A new movie and a known user
- A new user and new movie

Possible Solutions:

Use median of ratings for the known movie. Alternatively ask the new user to rate some movies and use cluster method to assign the user to a group and use the group average for the known movie instead.

Use genre, cast, directors etc to assign the new movie to a group of similar kind that the user has rated, then used the group average of the movie genre to rate the new movie.

A combination of two methods above, ask the new user to rate some movies then assign she/he to a group, then cluster the new movie and use average rating of user-group for the movie-cluster as the prediction.

### Collaborative Filtering with Stochastic Gradient Descent

Build a collaborative filtering model to predict ratings.

- Build the general architecture of a learning algorithm, including:
    - Encoding rating data
    - Initializing parameters
    - Calculating the cost function
    - Calculating gradient
    - Using an optimization algorithm (gradient descent)
    - Predicting on new data
- Putting it all together.

## Matrix Factorization with bias
We want to extend the Matrix Factorization model discussed in class to add a "bias" parameter for each user and another "bias" parameter for each movie.  For the problem in class we had the parameters matrix $U$ and $V$, we are adding $u^0$ which is a vector of dimension $n_u$ and $v^0$ which is a vector of dimension $n_m$. The equations

$$\hat{y}_{ij} = u_{0i} + v_{0j} + u_i \cdot v_j  $$ 
 
(a) How many weights (parameters) are we fitting for this problem?

(b) Write the gradient descent equations for this problem.

----

(a)
$$ n_u + n_m + (n_u + n_m)*K  $$ 

                                             where K is a hyperparameter

(b)
$$ u_{0i} = u_{0i}+\frac{2\eta}{N}\sum_{j: r_{i,j}=1}^{N} {(y_{ij} - u_{0i} - v_{0j} - u_i \cdot v_j)}  $$
$$ v_{0j} = v_{0i}+\frac{2\eta}{N}\sum_{i: r_{i,j}=1}^{N} {(y_{ij} - u_{0i} - v_{0j} - u_i \cdot v_j)}  $$
$$ u_{ik} = u_i+\frac{2\eta}{N}\sum_{j: r_{i,j}=1}^{N} {(y_{ij} - u_{0i} - v_{0j} - u_i \cdot v_j)}v_{jk}  $$
$$ v_{jk} = v_j+\frac{2\eta}{N}\sum_{i: r_{i,j}=1}^{N} {(y_{ij} - u_{0i} - v_{0j} - u_i \cdot v_j)}u_{ik}  $$

In [3]:
import numpy as np
import pandas as pd
from scipy import sparse
from cf import *

## Encoding rating data
Here are our very small subset of fake data to get us started.

In [4]:
# The first row says that user 1 reated movie 11 with a score of 4
!cat tiny_training.csv 

userId,movieId,rating
11,1,4
11,23,5
2,23,5
2,4,3
31,1,4
31,23,4
4,1,5
4,3,2
52,1,1
52,3,4
61,3,5
7,23,1
7,3,3


In [11]:
def proc_col(col):
    """Encodes a pandas column with values between 0 and n-1.
    where n = number of unique values
    """
    uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx[x] for x in col]), len(uniq)

In [16]:
def encode_data(df):
    """Encodes rating data with continous user and movie ids using 
    the helpful fast.ai function from above.
    
    Arguments:
      train_csv: a csv file with columns user_id,movie_id,rating 
    
    Returns:
      df: a dataframe with the encode data
      num_users
      num_movies  
    """
    df['userId'] = proc_col(df['userId'])[1]
    df['movieId'] = proc_col(df['movieId'])[1]
    num_users = proc_col(df['userId'])[2]
    num_movies = proc_col(df['movieId'])[2]
    return df, num_users, num_movies

In [17]:
# encode the training data
df = pd.read_csv("tiny_training.csv")
df, num_users, num_movies = encode_data(df)

In [18]:
df

,userId,movieId,rating
0,0,0,4
1,0,1,5
2,1,1,5
3,1,2,3
4,2,0,4
5,2,1,4
6,3,0,5
7,3,3,2
8,4,0,1
9,4,3,4


## Initializing parameters

In [24]:
def create_embedings(n, K):
    """ Create a numpy random matrix of shape n, K
    
    The random matrix should be initialized with uniform values in (0, 6/K)
    Arguments:
    
    Inputs:
    n: number of items/users
    K: number of factors in the embeding 
    
    Returns:
    emb: numpy array of shape (n, num_factors)
    """
    np.random.seed(3)
    emb = 6*np.random.random((n, K)) / K
    return emb

In [25]:
# here is an example on how the prediction matrix would look like with 7 users and 5 movies
np.dot(create_embedings(7,3), create_embedings(5,3).transpose())

array([[3.55790894, 4.69774849, 0.92361109, 1.58739544, 3.00593239],
       [4.69774849, 7.44656163, 1.18135616, 2.64524868, 4.74559066],
       [0.92361109, 1.18135616, 0.24548062, 0.34025121, 0.69616965],
       [1.58739544, 2.64524868, 0.34025121, 1.61561   , 2.41361975],
       [3.00593239, 4.74559066, 0.69616965, 2.41361975, 3.82505541],
       [2.02000808, 3.29656257, 0.43174569, 2.065911  , 3.07264619],
       [2.07691001, 3.02887291, 0.53270924, 1.02482544, 1.90251125]])

## Encoding Y as a sparse matrix
This code helps you encode a $Y$ as a sparse matrix from the dataframe. 

In [26]:
df = pd.read_csv("tiny_training.csv")
df, num_users, num_movies = encode_data(df)
Y = df2matrix(df, num_users, num_movies)

In [27]:
print(Y)

  (0, 0)	4
  (2, 0)	4
  (3, 0)	5
  (4, 0)	1
  (0, 1)	5
  (1, 1)	5
  (2, 1)	4
  (6, 1)	1
  (1, 2)	3
  (3, 3)	2
  (4, 3)	4
  (5, 3)	5
  (6, 3)	3


## Calculating the cost function

In [34]:
def df2matrix(df, nrows, ncols, column_name="rating"):
    """ Returns a sparse matrix constructed from a dataframe
    
    This code assumes the df has columns: MovieID,UserID,Rating
    """
    values = df[column_name].values
    ind_movie = df['movieId'].values
    ind_user = df['userId'].values
    return sparse.csc_matrix((values,(ind_user, ind_movie)),shape=(nrows, ncols))

In [35]:
def sparse_multiply(df, emb_user, emb_movie):
    """ This function returns U*V^T element wise multi by R as a sparse matrix.
    It avoids creating the dense matrix U*V^T
    """
    df["Prediction"] = np.sum(emb_user[df["userId"].values]*emb_movie[df["movieId"].values], axis=1)
    return df2matrix(df, emb_user.shape[0], emb_movie.shape[0], column_name="Prediction")

In [36]:
def cost(df, emb_user, emb_movie):
    """ Computes mean square error
    
    First compute prediction. Prediction for user i and movie j is
    emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      error(float): this is the MSE
    """
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0], column_name="rating")
    Y_hat = sparse_multiply(df,emb_user, emb_movie).toarray()
    N = Y.nnz
    Y = Y.toarray()
    error = np.power(Y-Y_hat, 2).sum()/N
    return error

In [37]:
emb_user = np.ones((num_users, 3))
emb_movie = np.ones((num_movies, 3))
error = cost(df, emb_user, emb_movie)

## Calculating gradient

In [38]:
K = 3
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
grad_user, grad_movie = gradient(df, Y, emb_user, emb_movie)

In [40]:
def finite_difference(df, emb_user, emb_movie, ind_u=None, ind_m=None, k=None):
    """ Computes finite difference on MSE(U, V).
    This function is used for testing the gradient function. 
    """
    e = 0.000000001
    c1 = cost(df, emb_user, emb_movie)
    K = emb_user.shape[1]
    x = np.zeros_like(emb_user)
    y = np.zeros_like(emb_movie)
    if ind_u is not None:
        x[ind_u][k] = e
    else:
        y[ind_m][k] = e
    c2 = cost(df, emb_user + x, emb_movie + y)
    return (c2 - c1)/e

In [41]:
user=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_u=user, k=i) for i in range(K)])
assert(np.all(np.abs(grad_user[user] - approx) < 0.0001))

In [42]:
movie=1
approx = np.array([finite_difference(df, emb_user, emb_movie, ind_m=movie, k=i) for i in range(K)])
assert(np.all(np.abs(grad_movie[movie] - approx) < 0.0001))

## Gradient descent with momentum

In [43]:
def gradient(df, Y, emb_user, emb_movie):
    """ Computes the gradient.
    
    First compute prediction. Prediction for user i and movie j is
    emb_user[i]*emb_movie[j]
    
    Arguments:
      df: dataframe with all data or a subset of the data
      Y: sparse representation of df
      emb_user: embedings for users
      emb_movie: embedings for movies
      
    Returns:
      d_emb_user
      d_emb_movie
    """
    Y_hat = sparse_multiply(df, emb_user, emb_movie)
    diff = Y - Y_hat
    d_emb_user = -2/Y.nnz * diff * emb_movie
    d_emb_movie = -2/Y.nnz * diff.transpose() * emb_user
    return d_emb_user, d_emb_movie

In [86]:
# use a for loop to iterate through gradient descent
def gradient_descent(df, emb_user, emb_movie, iterations=100, learning_rate=0.01, df_val=None):
    """ Computes gradient descent with momentum (0.9) for a number of iterations.
    
    Prints training cost and validation cost (if df_val is not None) every 50 iterations.
    
    Returns:
    emb_user: the trained user embedding
    emb_movie: the trained movie embedding
    """
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    momentum = 0.9 
    gradient_user_0, gradient_movie_0 = 0,0 
    for i in range(iterations):   
        gradient_user, gradient_movie = gradient(df, Y, emb_user, emb_movie)
        gradient_user_0 = momentum * gradient_user_0 + (1 - momentum) * gradient_user
        gradient_movie_0 = momentum * gradient_movie_0 + (1 - momentum) * gradient_movie
        emb_user = emb_user - learning_rate * gradient_user_0
        emb_movie = emb_movie - learning_rate * gradient_movie_0
        if i%50 == 0:
            print("training cost:", cost(df, emb_user, emb_movie))
            if df_val != None:
                print("validation cost:", cost(df_val, emb_user, emb_movie))
    return emb_user, emb_movie

emb_user = create_embedings(num_users, 3)
emb_movie = create_embedings(num_movies, 3)
emb_user, emb_movie = gradient_descent(df, emb_user, emb_movie, iterations=200, learning_rate=0.01,df_val = None)

training cost: 4.75400712709481
training cost: 1.9576510642217757
training cost: 1.0458015647845733
training cost: 0.7333410655387986


In [87]:
train_mse = cost(df, emb_user, emb_movie)

## Predicting on new data
Now we should write a function that given new data is able to predict ratings. First we write a function that encodes new data. If a new user or item is present that row should be remove. Collaborative Filtering is not good at handling new users or new items. To help with this task, you could write a an auxiliary function similar to `proc_col`.

In [88]:
def encode_new_data(df_val, df_train):
    """ Encodes df_val with the same encoding as df_train.
    Returns:
    df_val: dataframe with the same encoding as df_train
    """
    user = proc_col(df_train['userId'])[0]
    movie = proc_col(df_train['movieId'])[0]
    dup_userid = set(df_train['userId'])&set(df_val['userId'])
    dup_movieid = set(df_train['movieId'])&set(df_val['movieId'])
    df_val = df_val.iloc[np.where(df_val['userId'].isin(list(dup_userid)))]
    df_val = df_val.iloc[np.where(df_val['movieId'].isin(list(dup_movieid)))]
    df_val['userId']= np.array([user[i] for i in df_val['userId']])
    df_val['movieId']= np.array([movie[i] for i in df_val['movieId']])
    return df_val

In [89]:
df_t = pd.read_csv("tiny_training.csv")
df_v = pd.read_csv("tiny_val.csv")
df_v = encode_new_data(df_v, df_t)

In [90]:
len(df_v.userId.unique())

2

In [91]:
len(df_v) == 2

True

## Putting it all together
For this part you should get data from here
`wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip`

In [92]:
# sorting by timestamp
path = "ml-latest-small/"
data = pd.read_csv(path + "ratings.csv")
data = data.sort_values(by=['timestamp'])

In [93]:
data.head()

,userId,movieId,rating,timestamp
66719,429,595,5.0,828124615
66716,429,588,5.0,828124615
66717,429,590,5.0,828124615
66718,429,592,5.0,828124615
66712,429,432,3.0,828124615


In [94]:
m = int(data.shape[0]*0.8)
data.shape, m

((100836, 4), 80668)

In [95]:
train = data[:m].copy()
val = data[m:].copy()
df_train, num_users, num_movies = encode_data(train.copy())
df_val = encode_new_data(val.copy(), train.copy())
print(len(val), len(df_val))

20168 1311


In [96]:
# use a for loop to iterate through gradient descent
def gradient_descent(df, emb_user, emb_movie, iterations=100, learning_rate=0.01, df_val=None):
    """ Computes gradient descent with momentum (0.9) for a number of iterations.
    
    Prints training cost and validation cost (if df_val is not None) every 50 iterations.
    
    Returns:
    emb_user: the trained user embedding
    emb_movie: the trained movie embedding
    """
    Y = df2matrix(df, emb_user.shape[0], emb_movie.shape[0])
    momentum = 0.9 
    gradient_user_0, gradient_movie_0 = 0,0 
    for i in range(iterations):   
        gradient_user, gradient_movie = gradient(df, Y, emb_user, emb_movie)
        gradient_user_0 = momentum * gradient_user_0 + (1 - momentum) * gradient_user
        gradient_movie_0 = momentum * gradient_movie_0 + (1 - momentum) * gradient_movie
        emb_user = emb_user - learning_rate * gradient_user_0
        emb_movie = emb_movie - learning_rate * gradient_movie_0
        if i%50 == 0:
            print("training cost:", cost(df, emb_user, emb_movie))
            if len(df_val) != 0:
                print("validation cost:", cost(df_val, emb_user, emb_movie))
    return emb_user, emb_movie

In [97]:
K = 50
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=2000,learning_rate=1, df_val=df_val)

training cost: 12.136404414670121
validation cost: 12.31060129604362
training cost: 9.613903397530715
validation cost: 10.353878141282694
training cost: 6.4835207737519855
validation cost: 8.351624227410642
training cost: 4.519311775145553
validation cost: 6.96925304302195
training cost: 3.487453177675793
validation cost: 6.135265489703386
training cost: 2.8460807656602114
validation cost: 5.541986970150605
training cost: 2.4112356461406055
validation cost: 5.063727564992293
training cost: 2.1010335242073634
validation cost: 4.659691742103505
training cost: 1.8708720440691
validation cost: 4.309203797485996
training cost: 1.6945041588558352
validation cost: 4.000288133654588
training cost: 1.5556622025742264
validation cost: 3.7253323894374977
training cost: 1.4438504233658032
validation cost: 3.479028498334428
training cost: 1.3520657516237267
validation cost: 3.2573929170314457
training cost: 1.2754943751352943
validation cost: 3.057277550962371
training cost: 1.2107358665675336
vali

In [98]:
train_mse = cost(df_train, emb_user, emb_movie)
val_mse = cost(df_val, emb_user, emb_movie)
print(train_mse, val_mse)

0.7113513270477627 1.1405994500591452


In [99]:
train_mse = cost(df_train, emb_user, emb_movie)

In [100]:
val_mse = cost(df_val, emb_user, emb_movie)

In [101]:
K = 50
emb_user = create_embedings(num_users, K)
emb_movie = create_embedings(num_movies, K)
emb_user, emb_movie = gradient_descent(df_train, emb_user, emb_movie, iterations=200, learning_rate=1, df_val=df_val)

training cost: 12.136404414670121
validation cost: 12.31060129604362
training cost: 9.613903397530715
validation cost: 10.353878141282694
training cost: 6.4835207737519855
validation cost: 8.351624227410642
training cost: 4.519311775145553
validation cost: 6.96925304302195


In [103]:
train_mse = cost(df_train, emb_user, emb_movie)
val_mse = cost(df_val, emb_user, emb_movie)
print(train_mse, val_mse)

3.503321206458503 6.148866866928117
